In [4]:
from urllib.request import urlopen  
from urllib.parse import urlparse  
from bs4 import BeautifulSoup   
from urllib  import request  
import urllib  
import re
import datetime as dt
import pandas as pd
import csv

data_retrieve=[]
def load_page(url):

    page = request.Request(url)
    proxy_support=urllib.request.ProxyHandler({'https':'10.172.49.230:3128'})
    opener=urllib.request.build_opener(proxy_support,urllib.request.HTTPHandler)
    urllib.request.install_opener(opener)
    page.add_header('User-Agent','Mozilla/5.0 (Windows NT 6.1; \
        WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/48.0.2564.116 Safari/537.36')

    html = request.urlopen(page).read()

    soup = BeautifulSoup(html,'html.parser')
    
    get_herf_link(soup)

def tieba_spider(Url,startPage,endPage):
    #saving the desired data in a CSV file.
    for i in range(startPage,endPage + 1):
        page = i 
        url = Url + str(page)
        load_page(url)
        print(url)
        print("--------Downloading page %d, please wait----------" %i)
    
    save_to_csv(data_retrieve)

def get_topic_name(html):
    title=[]
    posting=html.find_all('div')
    for each in posting:
        if each.find(class_='topic') is not None:
            topic=each.find(class_='topic').get_text()
    
            #print(topic)
            return topic


def get_topic_content(html):
    content=[]
    data=html.find_all('div',class_="forum_op")
    for each in data:
        result=each.find(class_='content').get_text()
        
        return result


def get_reply_num(html):
    reply_num=[]
    replying=html.find_all('div', class_='topicstats')
    for each in replying:
        reply=each.find(class_='topicstats_value').get_text()
        if '@' in reply:
            continue
        else:
            #print(reply)
            return reply

def get_topic_post_time(html):  
    table=html.find_all('span', class_='date')
    for each in table:
        result=each['data-timestamp']
        #print(each['data-timestamp'])
    
        return result

def get_herf_link(html):
    posted_time=[]
    post_time=html.find_all('a', class_="forum_topic_overlay")
    #print(post_time)
    
    for each in post_time:
        second_link=each.get('href', None)
        load_next_page(second_link)
    
def load_next_page(url):
    global data_retrieve
    page = request.Request(url)
    proxy_support=urllib.request.ProxyHandler({'https':'10.172.49.230:3128'})
    opener=urllib.request.build_opener(proxy_support,urllib.request.HTTPHandler)
    urllib.request.install_opener(opener)
    page.add_header('User-Agent','Mozilla/5.0 (Windows NT 6.1; \
        WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/48.0.2564.116 Safari/537.36')

    html = request.urlopen(page).read()

    soup = BeautifulSoup(html,'html.parser')
    
    data=[get_topic_name(soup),
          get_topic_post_time(soup),
          get_topic_content(soup),
          get_reply_num(soup)]
    
    data_retrieve.append(data)
    #get_topic_name(soup)
    #get_topic_content(soup)
    #get_topic_post_time(soup)
    #get_reply_num(soup)
    
    
def replace(x):
    x = re.sub(re.compile('<img.*?>| {7}|'), "", x) #去除图像和7位空格
    x = re.sub(re.compile('<a.*?>|</a>'), "", x)  #去除链接
    x = re.sub(re.compile('<tr>|<div>|<div></p>'), '\n', x) #换行符替换成\n
    x = re.sub(re.compile('<td>'), '\t', x)  #制表符换位\t
    x = re.sub(re.compile('<p.*?>') , "\n  ", x) #段落符换位\n和两个空格
    x = re.sub(re.compile('<br>|<br><br>'), '\n', x) #换行符或双换行符替换为\n
    x = re.sub(re.compile('<.*?>'), "", x) #去掉其他符号
    return x.strip()

def save_to_csv(data):
    with open(r"D:\dataquest\PUBG\PUBG_analyzation1.csv",'w',newline='', encoding='utf-8') as csvFile:
        csvFile.write('\xEF\xBB\xBF') 
        writer =  csv.writer(csvFile)
        writer.writerow(('topic_title','posting_time', 'content','reply_num')) 
        for each in data:
            writer.writerow(each)

if __name__ == '__main__':
    Url = "http://steamcommunity.com/app/578080/discussions/?fp="
    startPage = 1
    endPage = 100
    tieba_spider(Url, startPage, endPage)
    
    print("-------------------Complete--------------------")
    
    

http://steamcommunity.com/app/578080/discussions/?fp=1
--------Downloading page 1, please wait----------


TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import datetime as dt
PUBG=pd.read_csv('PUBG_analyzation.csv')                 

#cleanning the data
def replace(x):
    x = re.sub(re.compile('<img.*?>| {7}|'), "", x) #去除图像和7位空格
    x = re.sub(re.compile('<a.*?>|</a>'), "", x)  #去除链接
    x = re.sub(re.compile('<tr>|<div>|<div></p>'), '\n', x) #换行符替换成\n
    x = re.sub(re.compile('<td>'), '\t', x)  #制表符换位\t
    x = re.sub(re.compile('<p.*?>') , "\n  ", x) #段落符换位\n和两个空格
    x = re.sub(re.compile('<br>|<br><br>'), '\n', x) #换行符或双换行符替换为\n
    x = re.sub(re.compile('<.*?>'), "", x) #去掉其他符号
    return x.strip()


clean_PUBG=PUBG.dropna(thresh=2)
PUBG40000=clean_PUBG[0:40000]

#topic cleaning
clean_title=PUBG40000['topic_title']
title_clean=[]
#clean_PUBG['topic_title']=clean_PUBG['topic_title'].apply(replace)
#clean_PUBG['reply_num']=clean_PUBG['reply_num'].apply(replace)
for each in clean_title:
    each=str(each)
    title = each.replace('\r\n\t\t\t\t\t\t\t\t\t', " ")
    title_clean.append(title)
PUBG40000['topic_title']=title_clean
PUBG40000


In [3]:
data_retrieve

[['\r\n\t\tNew reports/appeals system\t',
  '1514983552',
  '\r\n\t\tHi players,In an effort to improve our reporting/ban appeals systems, we have moved over to a new website where a dedicated team will be reviewing all submissions. Please use the link below for all reports and ban appeals:https://pubgsupport.zendesk.com/\n',
  '0'],
 ['\r\n\t\tPUBG Steam Hub Discussion Boards Update\t',
  '1506591884',
  '\r\n\t\tAfter a lengthy discussion we have decided that we will no-longer be actively attending to the PLAYERUNKNOWN’S BATTLEGROUNDS Steam Community forums. There are multiple reasons for this but the decision ultimately comes down to the fact that we are unable to work efficiently due to the moderation systems, or lack thereof, put in place by Steam. As our game has grown in popularity we have found that the amount of spam, rage posts, and other content that needs to be moderated, has increased to the point where we are unable to do so without full moderation tools found on every ot